<div style="color:red;background-color:black">
Diamond Light Source
<br style="color:red;background-color:antiquewhite"><h1>C Programming: Shared Libraries</h1><br>
©2000-21 Chris Seddon 
</div>

# 1: Build the library (linker fails)
Shared libraries are loaded into virtual memory by executable programs when they start. This enables many executables to share the same library code at run time.  This in turn means the shared library code does not need to be linked to the excutable (unlike with static libraries) thereby greatly reducing the size of the executable.  

We begin by compiling and linking the shared library code using the <b>-shared</b><a href="http://www.microhowto.info/howto/build_a_shared_library_using_gcc.html#idp25488"><b>-fPIC</b></a> options of gcc.  

However, when the executable is linked, the linker will not know where the shared library is located.

In [1]:
cd mylib
gcc -shared -fPIC -o libf.so f1.c f2.c f3.c

cd ../src
gcc f_main.c -o f.exe
cd ..

/usr/bin/ld: /tmp/cc774ncq.o: in function `main':
f_main.c:(.text+0xe): undefined reference to `f1'
/usr/bin/ld: f_main.c:(.text+0x18): undefined reference to `f2'
/usr/bin/ld: f_main.c:(.text+0x22): undefined reference to `f3'
collect2: error: ld returned 1 exit status


## 2: Place the shared library in the default location (/usr/lib)

We can locate our library with other system libraries by copying it to `/usr/lib`.  Note, however, you will need `sudo` privileges to do that.  The kernel needs to be informed of the location of the new shared library, so the kernel cache needs updating.

Unfortunately, the Jupyter bash kernel we are using does support reading on stdin, so we can't use `sudo` in our notebook.  

Since you probably don't have sudo pivileges we'll defer how this works until the end of this tutorial.

## 3: Using LD_LIBRARY_PATH (works but fragile)
If you can't use sudo, then one alternative is to use the `LD_LIBRARY_PATH` environment variable to locate the shared library.
This method works, but is fragile (the environment variable must be set correctly).  

We begin by compiling the library code:

In [2]:
cd mylib
gcc -shared -fPIC -o libf.so f1.c f2.c f3.c
cd ..

Next we setup the `LD_LIBRARY_PATH` relative to directory in which we compile our main program.  Then we can compile and run our executable:

In [3]:
cd src
export LD_LIBRARY_PATH=../mylib
gcc f_main.c -L../mylib -lf -o f.exe

# run the program
f.exe

cd ..

This is f1()
This is f2()
This is f3()


We can check where our shared library resides, by using `ldd`:

In [4]:
cd src

# check shared library dependencies
ldd f.exe
cd ..

	linux-vdso.so.1 (0x00007ffd50cab000)
	libf.so => ../mylib/libf.so (0x00007f3e5a948000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007f3e5a728000)
	/lib64/ld-linux-x86-64.so.2 (0x00007f3e5a954000)


## 4: Build Shared Library with RPATH (most reliable)
Rather than using `LD_LIBRAY_PATH` to locate the library, we can embed the path to the library in the executable as a relative path.  This involves talking to the linker directly.  Recall `gcc` is a macro that calls the compiler and the linker.  

To pass parameters to the linker we use the `-Wl,` option to `gcc`.

In [5]:
cd src
gcc f_main.c -Wl,-rpath=../mylib -L../mylib -lf -o f.exe
f.exe
ldd f.exe
cd ..

This is f1()
This is f2()
This is f3()
	linux-vdso.so.1 (0x00007ffc5cafe000)
	libf.so => ../mylib/libf.so (0x00007ff6213ad000)
	libc.so.6 => /lib/x86_64-linux-gnu/libc.so.6 (0x00007ff62118d000)
	/lib64/ld-linux-x86-64.so.2 (0x00007ff6213b9000)


Finally, let's look at info stored in the shared object.  Firstly use the `file` command:

In [6]:
cd mylib
file libf1.so
cd ..

libf1.so: ELF 64-bit LSB shared object, x86-64, version 1 (SYSV), dynamically linked, BuildID[sha1]=c88c731b59566694eb158ec9fbbf01e7cb3bbc70, not stripped


Secondly, use the `nm` command.  Note that we have not compiled with `-g` so debugging symbols will be missing:

In [7]:
cd mylib
nm libf1.so
cd ..

0000000000201028 B __bss_start
0000000000201028 b completed.7698
                 w __cxa_finalize@@GLIBC_2.2.5
0000000000000530 t deregister_tm_clones
00000000000005c0 t __do_global_dtors_aux
0000000000200e18 d __do_global_dtors_aux_fini_array_entry
0000000000201020 d __dso_handle
0000000000200e20 d _DYNAMIC
0000000000201028 D _edata
0000000000201030 B _end
000000000000060a T f1
0000000000000620 T _fini
0000000000000600 t frame_dummy
0000000000200e10 d __frame_dummy_init_array_entry
00000000000006d8 r __FRAME_END__
0000000000201000 d _GLOBAL_OFFSET_TABLE_
                 w __gmon_start__
0000000000000638 r __GNU_EH_FRAME_HDR
00000000000004e0 T _init
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
                 U puts@@GLIBC_2.2.5
0000000000000570 t register_tm_clones
0000000000201028 d __TMC_END__


To complete this tutorial we return to the original idea where we locate our library with other system libraries by copying it to `/usr/lib`.  Recall, you will need `sudo` privileges to do that.  The kernel needs to be informed of the location of the new shared library, so the kernel cache needs updating.

Unfortunately, the Jupyter bash kernel we are using does support reading on stdin, so we can't use `sudo` in our notebook.  

However, if you do have `sudo` privileges you can copy the shared library to `/usr/lib` and then compile, link our main program and finally run it, by execution the following commands in a terminal:

In [ ]:
cd mylib
sudo cp libf.so /usr/lib

# update cache
sudo ldconfig

# compile, link and run
cd ../src
gcc f_main.c -lf -o f.exe
f.exe

Finally, let's remove the shared library from `/usr/lib`.  Again, we will need to do the following on the command line, outside the notebook:

In [ ]:
sudo rm /usr/lib/libf.so